# Function Calling

OpenAI API에서 제공하는 선택적으로 특정함수를 호출하도록 제안하는 기능  
API의 tools에 functions 파라미터로 사용가능한 스키마를 제공하면, GPT에 의해 해당 기능을 호출하는 형태로 파라미터를 답한다.  
함수 시그니처에 준수하는 JSON으로 응답하도록 미세조정되어있어 구조화된 데이터를 보다 안정적으로 얻을 수 있다.  

지원 모델

-   gpt-4o
-   gpt-4-turbo
-   gpt-3.5-turbo


In [1]:
from openai import OpenAI
import json

## 입력데이터

In [2]:
question = "머지 정렬이란 무엇인가?"
answer = "존 폰 노이만이 제안. 안정 정렬에 속하며, 분할 정복 알고리즘의 하나.\
          문제를 작은 2개의 문제로 분리하고 각각을 해결한 다음, 결과를 모아서 원래의 문제를 해결하는 전략.\
          시간복잡도는 O(NlogN)"

In [3]:
test_paper = "하나의 리스트를 두개의 균등한 크기로 분할을 하고,\
              분할된 부분 리스트들을 정렬을 한 다음에,\
              정렬된 부분 리스트들을 하나로 합쳐요"

## 스키마

정리된 아웃풋을 사용할 함수 형태

```python
def make_eval_sheet(score, feedback):
    """Make a sheet with score and feedback"""
```

Function Calling에 사용할 스키마 형태로 작성

In [4]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "make_eval_sheet",
            "description": "Make a sheet with score and feedback",
            "parameters": {
                "type": "object",
                "properties": {
                    "score": {
                        "type": "number",
                        "scope": "range(100)",
                        "description": "Scoring the solution based on the presentation of the problem and answers. 100 points perfect",
                    },
                    "feedback": {
                        "type": "string", 
                        "description": "Providing feedback on the solution based on the problem and answer. Answer in Korean",
                    },
                },
                "required": ["score"],
            },
        },
    }
]

## Function Calling 실행

In [5]:
eval_instruction = f"The problem statement is '{question}', with the correct answer being '{answer}'.\
                     The student responded with '{test_paper}'.\
                     Please provide a score and feedback to create a report."

messages = [{"role": "user", "content": eval_instruction}]

In [6]:
client = OpenAI()
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=0.4,
    tools=tools,
    tool_choice="auto",
)
response_message = response.choices[0].message
tool_calls = response_message.tool_calls
tool_calls

[ChatCompletionMessageToolCall(id='call_vN18ZsJcPwtn5nVAKxGUCmhK', function=Function(arguments='{"score":80,"feedback":"좋은 시도였지만, 머지 정렬에 대한 설명이 부족합니다. 정확한 설명을 더 추가하여 완벽한 점수를 받을 수 있을 것입니다."}', name='make_eval_sheet'), type='function')]

아웃풋 정리

In [7]:
if tool_calls:
    for tool_call in tool_calls:
        function_args = json.loads(tool_call.function.arguments)
        print("점수: ", function_args.get("score"))
        print("피드백: ", function_args.get("feedback"))

점수:  80
피드백:  좋은 시도였지만, 머지 정렬에 대한 설명이 부족합니다. 정확한 설명을 더 추가하여 완벽한 점수를 받을 수 있을 것입니다.
